# Callback回调机制

[![下载Notebook](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_callback.ipynb)&emsp;
[![下载样例代码](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_callback.py)&emsp;
[![查看源文件](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/intermediate/callback.ipynb)

在深度学习训练过程中，为及时掌握网络模型的训练状态、实时观察网络模型各参数的变化情况和实现训练过程中用户自定义的一些操作，MindSpore提供了Callback回调机制。

Callback回调机制一般用在网络模型训练过程`Model.train`中，MindSpore会按照Callback列表`callbacks`顺序执行回调函数，用户可以通过配置不同的回调函数来实现不同功能。

## 常用的内置回调函数

下面以基于MNIST数据集训练LeNet-5网络模型为例，介绍几种常用的MindSpore内置回调函数。

> 更多内置回调函数信息及使用方式请参考[API文档](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.train.html#mindspore-train-callback)。

- ModelCheckpoint，保存训练后的模型和网络参数，方便进行再推理或再训练。
- LossMonitor，监控训练过程中的损失函数值Loss变化情况。
- TimeMonitor，监控训练过程中每个epoch、每个step的运行时间。

!!!zhaoyu，LossMonitor和TimeMonitor在套件中用统一接口展示，另外套件针对推理还提供了接口，请展示。

!!!zhaoyu,callback建议都拆出来成一个目录，里面有三个内容，1个ModelCheckpoint，一个套件的Monitor，一个也是套件里面有的接口：https://mindspore.cn/docs/programming_guide/zh-CN/master/evaluate_the_model_during_training.html。

使用示例：

In [2]:
import mindspore.nn as nn
from mindspore.train import Model
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, TimeMonitor, LossMonitor

from mindvision.classification.dataset import Mnist
from mindvision.classification.models import lenet

download_train = Mnist(path="./mnist", split="train", batch_size=32, repeat_num=1, shuffle=True, resize=32, download=True)
dataset_train = download_train.run()

network = lenet(num_classes=10, pretrained=False)
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
net_opt = nn.Momentum(network.trainable_params(), learning_rate=0.01, momentum=0.9)
model = Model(network, loss_fn=net_loss, optimizer=net_opt, metrics={'acc'})

config_ck = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=10)
ckpoint = ModelCheckpoint(prefix="lenet", directory="./lenet", config=config_ck)

# !!!代码加上简单注释，特别是针对Monitor
model.train(1, dataset_train, callbacks=[ckpoint, LossMonitor(300), TimeMonitor(1)], dataset_sink_mode=False)

epoch: 1 step: 300, loss is 2.295940399169922
epoch: 1 step: 600, loss is 2.301466226577759
epoch: 1 step: 900, loss is 0.3619288206100464
epoch: 1 step: 1200, loss is 0.06575474143028259
epoch: 1 step: 1500, loss is 0.16204708814620972
epoch: 1 step: 1800, loss is 0.004415267147123814
epoch time: 16472.533 ms, per step time: 8.785 ms


!!!zhaoyu，套件针对推理还提供了接口，请展示

生成的Checkpoint文件目录结构如下：

```text
./lenet/
├── lenet-1_1875.ckpt # 保存参数文件
└── lenet-graph.meta # 编译后的计算图
```

## 自定义回调函数

MindSpore不仅有功能强大的内置回调函数，当用户有自己的特殊需求时，还可以基于`Callback`基类自定义回调函数。

回调机制可以把训练过程中的重要信息记录下来，通过把一个字典类型变量`RunContext.original_args()`传递给Callback对象，使得用户可以在各个自定义的Callback中获取到相关属性，执行自定义操作，也可以自定义其他变量传递给`RunContext.original_args()`对象。

`RunContext.original_args()`中的常用属性有：

- loss_fn：损失函数
- optimizer：优化器
- train_dataset：训练的数据集
- epoch_num：训练的epoch的数量
- batch_num：一个epoch中step的数量
- train_network：训练的网络
- cur_epoch_num：当前的epoch数
- cur_step_num：当前的step数
- parallel_mode：并行模式
- list_callback：所有的Callback函数
- net_outputs：网络的输出结果

通过下面两个场景，我们可以增加对自定义Callback回调函数功能的了解。

1. 实现在规定时间内终止训练。用户可以设定时间阈值，当训练时间达到这个阈值后就终止训练过程。

!!!zhaoyu，代码实现方式参考套件中的接口，这里的输出没有格式化，另外需要加上代码的解释，这里面用了很多变量。

In [3]:
# zhaoyu，注意代码的引用顺序
import time

from mindspore.train.callback import Callback, LossMonitor

class StopAtTime(Callback):
    def __init__(self, run_time):
        super(StopAtTime, self).__init__()
        self.run_time = run_time

    def begin(self, run_context):
        cb_params = run_context.original_args()
        cb_params.init_time = time.time()

        print("begin training, time is:", cb_params.init_time)

    def step_end(self, run_context):
        cb_params = run_context.original_args()
        epoch_num = cb_params.cur_epoch_num
        step_num = cb_params.cur_step_num
        loss = cb_params.net_outputs
        cur_time = time.time()

        if (cur_time - cb_params.init_time) > self.run_time:
            print("epoch: ", epoch_num, " step: ", step_num, " loss: ", loss)
            print("end training, time is:", cur_time)
            run_context.request_stop()

model.train(1, dataset_train, callbacks=[LossMonitor(200), StopAtTime(10)], dataset_sink_mode=False)

begin training, time is: 1646122014.3542275
epoch: 1 step: 200, loss is 0.10437318682670593
epoch: 1 step: 400, loss is 0.08411901444196701
epoch: 1 step: 600, loss is 0.16160571575164795
epoch: 1 step: 800, loss is 0.008495703339576721
epoch: 1 step: 1000, loss is 0.013355637900531292
epoch:  1  step:  1172  loss:  0.007238159
end training, time is: 1646122024.3574462


2. 实现当loss小于设定的阈值时，保存ckpt文件。

In [4]:
from mindspore import save_checkpoint
from mindspore.train.callback import Callback

# zhaoyu,请注意代码格式和注释，output我已改过来，参考套件的monitor补充对齐格式。
class SaveCallback(Callback):

    def step_end(self, run_context):
        cb_params = run_context.original_args()
        loss = cb_params.net_outputs.asnumpy()

        if loss < 5e-4:
            file_name = str(cb_params.cur_epoch_num) + "_" + str(cb_params.cur_step_num) + ".ckpt"
            save_checkpoint(save_obj=cb_params.train_network, ckpt_file_name=file_name)

            print(f"Saved checkpoint, loss: {loss}, current step num: {cb_params.cur_step_num}.")

model.train(1, dataset_train, callbacks=[SaveCallback()], dataset_sink_mode=False)

Save the checkpoint, the loss is:  0.0004997954 the current step num is:  191
Save the checkpoint, the loss is:  0.00022582882 the current step num is:  268
Save the checkpoint, the loss is:  0.000142571 the current step num is:  269
Save the checkpoint, the loss is:  0.00024890315 the current step num is:  381
Save the checkpoint, the loss is:  0.0003373801 the current step num is:  932
Save the checkpoint, the loss is:  0.00048165582 the current step num is:  1108
Save the checkpoint, the loss is:  0.00036466357 the current step num is:  1237
Save the checkpoint, the loss is:  0.00012776905 the current step num is:  1687
Save the checkpoint, the loss is:  0.00040109223 the current step num is:  1690


保存目录结构如下：

```text
./
├── 1_191.ckpt
├── 1_268.ckpt
├── 1_269.ckpt
├── 1_381.ckpt
├── 1_932.ckpt
├── 1_1108.ckpt
├── 1_1237.ckpt
├── 1_1687.ckpt
├── 1_1690.ckpt
```